# ADIVINA LA FRASE

Implementad un algoritmo genético que encuentre la frase (con caracteres ASCII) introducida
al ejecutar el algoritmo.
- El algoritmo genético no conoce la frase, pero la función de fitness sí.
- Los caracteres ASCII se pueden representar como valores enteros del 0 al 255. En
Python: chr(65) == ‘A’, ord(‘A’) == 65


In [1]:
import random

class GeneticAlgorithm(object):
    def __init__(self, genetics):
        self.genetics = genetics
        pass

    def run(self):
        population = self.genetics.initial()
        while True:
            fits_pops = [(self.genetics.fitness(ch),  ch) for ch in population]
            if self.genetics.check_stop(fits_pops): break
            population = self.next(fits_pops)
            pass
        return population

    def next(self, fits):
        parents_generator = self.genetics.parents(fits)
        size = len(fits)
        nexts = []
        while len(nexts) < size:
            parents = next(parents_generator)
            cross = random.random() < self.genetics.probability_crossover()
            children = self.genetics.crossover(parents) if cross else parents
            for ch in children:
                mutate = random.random() < self.genetics.probability_mutation()
                nexts.append(self.genetics.mutation(ch) if mutate else ch)
                pass
            pass
        return nexts[0:size]
    pass

class GeneticFunctions(object):
    def probability_crossover(self):
        r"""returns rate of occur crossover(0.0-1.0)"""
        return 1.0

    def probability_mutation(self):
        r"""returns rate of occur mutation(0.0-1.0)"""
        return 0.0

    def initial(self):
        r"""returns list of initial population
        """
        return []

    def fitness(self, chromosome):
        r"""returns domain fitness value of chromosome
        """
        return len(chromosome)

    def check_stop(self, fits_populations):
        r"""stop run if returns True
        - fits_populations: list of (fitness_value, chromosome)
        """
        return False

    def parents(self, fits_populations):
        r"""generator of selected parents
        """
        gen = iter(sorted(fits_populations))
        while True:
            f1, ch1 = next(gen)
            f2, ch2 = next(gen)
            yield (ch1, ch2)
            pass
        return

    def crossover(self, parents):
        r"""breed children
        """
        return parents

    def mutation(self, chromosome):
        r"""mutate chromosome
        """
        return chromosome
    pass

if __name__ == "__main__":
    """
    example: Mapped guess prepared Text
    """
    class GuessText(GeneticFunctions):
        def __init__(self, target_text,
                     limit=2000, size=400,
                     prob_crossover=0.9, prob_mutation=0.2):
            self.target = self.text2chromo(target_text)
            self.counter = 0

            self.limit = limit
            self.size = size
            self.prob_crossover = prob_crossover
            self.prob_mutation = prob_mutation
            pass

        # GeneticFunctions interface impls
        def probability_crossover(self):
            return self.prob_crossover

        def probability_mutation(self):
            return self.prob_mutation

        def initial(self):
            return [self.random_chromo() for j in range(self.size)]

        def fitness(self, chromo):
            # larger is better, matched == 0
            return -sum(abs(c - t) for c, t in zip(chromo, self.target))

        def check_stop(self, fits_populations):
            self.counter += 1
            if self.counter % 10 == 0:
                best_match = list(sorted(fits_populations))[-1][1]
                fits = [f for f, ch in fits_populations]
                best = max(fits)
                worst = min(fits)
                ave = sum(fits) / len(fits)
                print(
                    "[G %3d] score=(%4d, %4d, %4d): %r" %
                    (self.counter, best, ave, worst,
                     self.chromo2text(best_match)))
                pass
            return self.counter >= self.limit

        def parents(self, fits_populations):
            while True:
                father = self.tournament(fits_populations)
                mother = self.tournament(fits_populations)
                yield (father, mother)
                pass
            pass

        def crossover(self, parents):
            father, mother = parents
            index1 = random.randint(1, len(self.target) - 2)
            index2 = random.randint(1, len(self.target) - 2)
            if index1 > index2: index1, index2 = index2, index1
            child1 = father[:index1] + mother[index1:index2] + father[index2:]
            child2 = mother[:index1] + father[index1:index2] + mother[index2:]
            return (child1, child2)

        def mutation(self, chromosome):
            index = random.randint(0, len(self.target) - 1)
            vary = random.randint(-5, 5)
            mutated = list(chromosome)
            mutated[index] += vary
            return mutated

        # internals
        def tournament(self, fits_populations):
            alicef, alice = self.select_random(fits_populations)
            bobf, bob = self.select_random(fits_populations)
            return alice if alicef > bobf else bob

        def select_random(self, fits_populations):
            return fits_populations[random.randint(0, len(fits_populations)-1)]

        def text2chromo(self, text):
            return [ord(ch) for ch in text]
        def chromo2text(self, chromo):
            return "".join(chr(max(1, min(ch, 255))) for ch in chromo)

        def random_chromo(self):
            return [random.randint(1, 255) for i in range(len(self.target))]
        pass

    GeneticAlgorithm(GuessText("Estamos hablando de tonterias muy grandes!")).run()
    pass

[G  10] score=(-1373, -1890, -2379): "7KrQk\x9a\x9e>\x8f\x8b3(v@wZ'\x87_\x03!ÅsWSY¢t^\tBeB,Z¤\x98V\x947/\x1a"
[G  20] score=(-819, -1097, -1460): '6S2aku\x8d\x05_\x8b_RNHwj%h|1W>|b~\x8cptx\x06RkU%Yt}{{\x84\x800'
[G  30] score=(-525, -668, -842): 'I\x7fuakut\x16_\x8e\\RIgVZ%hc1\x88wswD\x8aqtn\x12qpU!]o}{{]\x92\x19'
[G  40] score=(-400, -465, -543): 'I\x7fuapuu\x16oh_RfgY{$hc1\x88koba[nmn,qpU!Wtg{{]\x92\x19'
[G  50] score=(-315, -352, -390): "I\x7fudkuu'__^SfdY{ hc1yiotc[nmn)qp\x8e!]rc{j]\x92\x19"
[G  60] score=(-276, -298, -323): 'I\x7fuakit\x16kh_Vfg]k ec1yootc[nmn)qp\x8e!]scuf]\x92\x19'
[G  70] score=(-228, -255, -279): "6yuakut'l__Vfg]k ec1wootc[ihn)qu\x8a!]rcwf]t-"
[G  80] score=(-202, -220, -237): "6|vakuu'j_cVfg^k ec1tnotc`ims)mu\x87!brcwfft-"
[G  90] score=(-164, -187, -208): '?\x7fuakut"j__Xaebj ec,uootcdihs)mu\x81!bscwfft-'
[G 100] score=(-137, -152, -171): '>yuakus"i_cXfkbo dc,uootcdihs$mu\x82!frcufat*'
[G 110] score=(-110, -123, -139): ':tuakpt"k__\\akbo dc\'uootdgihs$mu\x82!

[G 1060] score=(   0,    0,   -6): 'Estamos hablando de tonterias muy grandes!'
[G 1070] score=(   0,    0,   -5): 'Estamos hablando de tonterias muy grandes!'
[G 1080] score=(   0,    0,   -9): 'Estamos hablando de tonterias muy grandes!'
[G 1090] score=(   0,    0,   -7): 'Estamos hablando de tonterias muy grandes!'
[G 1100] score=(   0,    0,   -9): 'Estamos hablando de tonterias muy grandes!'
[G 1110] score=(   0,    0,   -8): 'Estamos hablando de tonterias muy grandes!'
[G 1120] score=(   0,    0,   -6): 'Estamos hablando de tonterias muy grandes!'
[G 1130] score=(   0,    0,   -7): 'Estamos hablando de tonterias muy grandes!'
[G 1140] score=(   0,    0,   -9): 'Estamos hablando de tonterias muy grandes!'
[G 1150] score=(   0,    0,   -6): 'Estamos hablando de tonterias muy grandes!'
[G 1160] score=(   0,    0,   -5): 'Estamos hablando de tonterias muy grandes!'
[G 1170] score=(   0,    0,   -5): 'Estamos hablando de tonterias muy grandes!'
[G 1180] score=(   0,    0,   -7): 'Esta